In [ ]:
!cp -r /content/drive/MyDrive/Magang /content

In [ ]:
# hci_pipeline.py
# ============================================================
# Home Credit – End-to-end pipeline (Colab/CPU friendly)
# Langkah: Data hygiene -> FE -> Aggregasi satelit -> 5-fold CV
#          -> LGBM (+ optional CatBoost) -> Blend -> Calibration
#          -> Refit full -> submission.csv
# ============================================================

import os
import math
import json
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

# Model & metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.isotonic import IsotonicRegression

# LightGBM
import lightgbm as lgb

# CatBoost (opsional, jika tersedia)
try:
    from catboost import CatBoostClassifier, Pool
    HAS_CATBOOST = True
except Exception:
    HAS_CATBOOST = False

RANDOM_STATE = 42
N_SPLITS = 5
EARLY_STOPPING_ROUNDS = 200

# ============================================================
# 0) PATHS (sesuaikan sesuai lokasi file Anda)
# ============================================================
DATA_DIR = "/content/Magang"

PATH_APP_TRAIN  = os.path.join(DATA_DIR, "application_train.csv")
PATH_APP_TEST   = os.path.join(DATA_DIR, "application_test.csv")
PATH_BUREAU     = os.path.join(DATA_DIR, "bureau.csv")
PATH_BB         = os.path.join(DATA_DIR, "bureau_balance.csv")
PATH_PREV       = os.path.join(DATA_DIR, "previous_application.csv")
PATH_POS        = os.path.join(DATA_DIR, "POS_CASH_balance.csv")
PATH_CC         = os.path.join(DATA_DIR, "credit_card_balance.csv")
PATH_INST       = os.path.join(DATA_DIR, "installments_payments.csv")  # nama umum; ganti bila berbeda
# Catatan: user mengunggah credit_installments_payments.py/.ipynb; dataset standar bernama "installments_payments.csv".
# Jika file Anda berbeda, ubah PATH_INST.

OUT_DIR = "/content/hci_outputs"
os.makedirs(OUT_DIR, exist_ok=True)


In [ ]:
# ============================================================
# 1) UTILITIES
# ============================================================

def safe_log1p(series):
    """Log1p untuk kolom amount yang sangat skewed (handle nilai negatif/NaN)."""
    s = pd.to_numeric(series, errors="coerce")
    s = s.clip(lower=0)  # floor ke 0 untuk jaga-jaga
    return np.log1p(s)

def winsorize_series(s, lower_q=0.0, upper_q=0.995):
    x = pd.to_numeric(s, errors="coerce")
    lo = x.quantile(lower_q)
    hi = x.quantile(upper_q)
    return x.clip(lower=lo, upper=hi)

def ratio(a, b, eps=1e-6):
    a = pd.to_numeric(a, errors="coerce")
    b = pd.to_numeric(b, errors="coerce").replace(0, np.nan)
    return a / (b + eps)

def ensure_sk_id_curr(df, left_df=None, left_key=None, right_key=None):
    """
    Pastikan df punya SK_ID_CURR.
    - Jika df sudah punya, kembalikan df.
    - Jika tidak, coba isi dari previous_application (via SK_ID_PREV) atau bureau (via SK_ID_BUREAU).
    Param:
      left_df: dataframe referensi (previous_application atau bureau)
      left_key/right_key: pasangan kolom untuk merge (mis. left_key='SK_ID_PREV', right_key='SK_ID_PREV')
    """
    if df is None or len(df) == 0:
        return df
    if "SK_ID_CURR" in df.columns:
        return df
    if left_df is not None and left_key is not None and right_key is not None:
        if left_key in left_df.columns and right_key in df.columns:
            temp = df.merge(left_df[[left_key, "SK_ID_CURR"]].drop_duplicates(),
                            left_on=right_key, right_on=left_key, how="left")
            temp.drop(columns=[left_key], inplace=True, errors="ignore")
            return temp
    # Jika tetap gagal, kembalikan apa adanya (nanti agregasi akan memfilter NaN)
    return df

def percentify_counts(counts):
    """Ubah value_counts STATUS menjadi proporsi."""
    total = counts.sum(axis=1).replace(0, np.nan)
    return counts.div(total, axis=0).add_prefix("SHARE_")

def make_ohe_small_cardinality(df, max_card=20, exclude_cols=None):
    """
    One-hot hanya untuk kategori dengan kardinalitas kecil agar tidak meledak.
    Kembalikan df_num (numeric) + OHE kecil.
    """
    if exclude_cols is None:
        exclude_cols = []
    out = df.copy()

    cat_cols = [c for c in out.columns
                if (out[c].dtype == "object" or str(out[c].dtype) == "category")
                and c not in exclude_cols]
    small_cats = []
    for c in cat_cols:
        if out[c].nunique(dropna=True) <= max_card:
            small_cats.append(c)

    if small_cats:
        ohe = pd.get_dummies(out[small_cats], prefix=small_cats, dummy_na=False)
        out = pd.concat([out.drop(columns=small_cats), ohe], axis=1)

    # Pastikan semua numeric-friendly
    for col in out.columns:
        if out[col].dtype == "object":
            out[col] = out[col].astype("category")

    return out



In [ ]:
# ============================================================
# 2) LOAD application_train/test + basic hygiene & fitur inti
# ============================================================

print("[LOAD] application_train/test ...")
app_train = pd.read_csv(PATH_APP_TRAIN)
app_test  = pd.read_csv(PATH_APP_TEST)

TARGET = app_train["TARGET"].values
SK_ID_TRAIN = app_train["SK_ID_CURR"].values
SK_ID_TEST  = app_test["SK_ID_CURR"].values

# Gabung sementara untuk FE konsisten
app_train["__IS_TRAIN__"] = 1
app_test["__IS_TRAIN__"] = 0
app_all = pd.concat([app_train.drop(columns=["TARGET"]), app_test], axis=0, ignore_index=True)

# Hygiene: DAYS_EMPLOYED anomaly -> NaN
if "DAYS_EMPLOYED" in app_all.columns:
    app_all.loc[app_all["DAYS_EMPLOYED"] == 365243, "DAYS_EMPLOYED"] = np.nan

# Fitur rasio inti
def add_core_application_features(df):
    # --- Rasio inti ---
    if "AMT_CREDIT" in df.columns and "AMT_INCOME_TOTAL" in df.columns:
        df["CREDIT_INCOME_RATIO"] = ratio(df["AMT_CREDIT"], df["AMT_INCOME_TOTAL"])
    if "AMT_ANNUITY" in df.columns and "AMT_INCOME_TOTAL" in df.columns:
        df["ANNUITY_INCOME_RATIO"] = ratio(df["AMT_ANNUITY"], df["AMT_INCOME_TOTAL"])
    if "AMT_ANNUITY" in df.columns and "AMT_CREDIT" in df.columns:
        df["ANNUITY_CREDIT_RATIO"] = ratio(df["AMT_ANNUITY"], df["AMT_CREDIT"])

    # --- Umur & masa kerja ---
    if "DAYS_BIRTH" in df.columns:
        df["AGE_YEARS"] = -pd.to_numeric(df["DAYS_BIRTH"], errors="coerce") / 365.25
    if "DAYS_EMPLOYED" in df.columns:
        df["EMPLOYMENT_YEARS"] = -pd.to_numeric(df["DAYS_EMPLOYED"], errors="coerce") / 365.25

    # --- Transformasi amount (konversi numeric per kolom, log1p, winsorize) ---
    for col in ["AMT_INCOME_TOTAL", "AMT_CREDIT", "AMT_ANNUITY", "AMT_GOODS_PRICE"]:
        if col in df.columns:
            num = pd.to_numeric(df[col], errors="coerce")
            df[f"LOG1P_{col}"] = safe_log1p(num)
            df[col] = winsorize_series(num)

    # --- EXT_SOURCE_* (konversi per kolom dulu, baru mean/std baris) ---
    ext_cols = [c for c in df.columns if c.startswith("EXT_SOURCE_")]
    if ext_cols:
        ext_num = df[ext_cols].apply(pd.to_numeric, errors="coerce")
        df["EXT_SOURCE_MEAN"] = ext_num.mean(axis=1)
        df["EXT_SOURCE_STD"]  = ext_num.std(axis=1)

    # --- Flags kualitas data (pastikan numeric per kolom) ---
    for col in ["FLAG_MOBIL", "FLAG_EMP_PHONE", "FLAG_WORK_PHONE", "FLAG_CONT_MOBILE", "FLAG_PHONE", "FLAG_EMAIL"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # --- Hitung dokumen hilang (DOC*) ---
    doc_cols = [c for c in df.columns if c.startswith("FLAG_DOCUMENT_")]
    if doc_cols:
        docs_num = df[doc_cols].apply(pd.to_numeric, errors="coerce").fillna(0)
        # Asumsi FLAG_DOCUMENT_* = 1 jika dokumen ADA → dokumen hilang = 1 - flag
        df["DOC_MISSING_COUNT"] = (1 - docs_num).sum(axis=1)

    return df

app_all = add_core_application_features(app_all)

# Simpan kolom kategori yang akan di-OHE ringan
exclude_ohe_cols = ["SK_ID_CURR", "__IS_TRAIN__"]
app_all = make_ohe_small_cardinality(app_all, max_card=20, exclude_cols=exclude_ohe_cols)


[LOAD] application_train/test ...


In [ ]:
# ============================================================
# 3) LOAD tabel satelit & agregasi
# ============================================================

def aggregate_bureau_and_balance(bureau_path, bb_path):
    if not os.path.exists(bureau_path):
        return None
    bureau = pd.read_csv(bureau_path)

    # Agregasi dari bureau_balance ke bureau
    if os.path.exists(bb_path):
        bb = pd.read_csv(bb_path)
        if "SK_ID_BUREAU" in bb.columns and "STATUS" in bb.columns:
            # value_counts pivot → share per status
            status_ct = (bb
                         .groupby("SK_ID_BUREAU")["STATUS"]
                         .value_counts()
                         .unstack(fill_value=0))
            status_share = percentify_counts(status_ct)
            bureau = bureau.merge(status_share, left_on="SK_ID_BUREAU", right_index=True, how="left")

    # Agregasi ke level SK_ID_CURR
    # Feature contoh: jumlah pinjaman, share ACTIVE/CLOSED, overdue, max DPD
    agg_dict = {}

    if "CREDIT_ACTIVE" in bureau.columns:
        # share per status kredit aktif
        ct = (bureau.groupby("SK_ID_CURR")["CREDIT_ACTIVE"]
              .value_counts().unstack(fill_value=0))
        share = percentify_counts(ct)
        share.columns = [f"BUREAU_{c}" for c in share.columns]
        base = share
    else:
        base = bureau.groupby("SK_ID_CURR").size().to_frame("BUREAU_N_LOANS")

    # numeric aggs
    num_cols = []
    for c in ["AMT_CREDIT_MAX_OVERDUE", "AMT_CREDIT_SUM", "AMT_CREDIT_SUM_DEBT", "AMT_CREDIT_SUM_LIMIT", "AMT_CREDIT_SUM_OVERDUE",
              "DAYS_CREDIT", "CREDIT_DAY_OVERDUE", "CNT_CREDIT_PROLONG"]:
        if c in bureau.columns:
            num_cols.append(c)
            agg_dict[c] = ["mean", "max"]

    if num_cols:
        num_agg = bureau.groupby("SK_ID_CURR")[num_cols].agg(agg_dict)
        num_agg.columns = ["BUREAU_" + "_".join(col).upper() for col in num_agg.columns.ravel()]
        base = base.merge(num_agg, left_index=True, right_index=True, how="left")

    # if introduced STATUS shares from bb already merged to bureau rows:
    bb_share_cols = [c for c in bureau.columns if c.startswith("SHARE_")]
    if bb_share_cols:
        bb_agg = bureau.groupby("SK_ID_CURR")[bb_share_cols].mean()
        bb_agg.columns = ["BUREAU_BAL_" + c for c in bb_agg.columns]
        base = base.merge(bb_agg, left_index=True, right_index=True, how="left")

    base.reset_index(inplace=True)
    return base

def aggregate_previous(prev_path):
    if not os.path.exists(prev_path):
        return None
    prev = pd.read_csv(prev_path)

    # approval rate
    if "NAME_CONTRACT_STATUS" in prev.columns:
        prev["IS_APPROVED"] = (prev["NAME_CONTRACT_STATUS"] == "Approved").astype(int)
    else:
        prev["IS_APPROVED"] = np.nan

    agg_dict = {}
    for c in ["AMT_ANNUITY", "AMT_APPLICATION", "AMT_CREDIT", "AMT_DOWN_PAYMENT",
              "AMT_GOODS_PRICE", "RATE_DOWN_PAYMENT", "HOUR_APPR_PROCESS_START",
              "DAYS_DECISION", "CNT_PAYMENT"]:
        if c in prev.columns:
            agg_dict[c] = ["mean", "max"]

    base = prev.groupby("SK_ID_CURR").size().to_frame("PREV_N_APPS")
    if agg_dict:
        num_agg = prev.groupby("SK_ID_CURR")[list(agg_dict.keys())].agg(agg_dict)
        num_agg.columns = ["PREV_" + "_".join(col).upper() for col in num_agg.columns.ravel()]
        base = base.merge(num_agg, left_index=True, right_index=True, how="left")

    if "IS_APPROVED" in prev.columns:
        appr = prev.groupby("SK_ID_CURR")["IS_APPROVED"].mean().to_frame("PREV_APPROVAL_RATE")
        base = base.merge(appr, left_index=True, right_index=True, how="left")

    base.reset_index(inplace=True)
    return base

def aggregate_pos(pos_path, prev_path=None):
    if not os.path.exists(pos_path):
        return None
    pos = pd.read_csv(pos_path)

    # Pastikan punya SK_ID_CURR
    prev_df = None
    if prev_path and os.path.exists(prev_path):
        prev_df = pd.read_csv(prev_path)[["SK_ID_PREV", "SK_ID_CURR"]]
    pos = ensure_sk_id_curr(pos, left_df=prev_df, left_key="SK_ID_PREV", right_key="SK_ID_PREV")

    if "SK_ID_CURR" not in pos.columns:
        return None

    # DPD max/mean, count months, share active?
    agg_dict = {}
    for c in ["SK_DPD", "SK_DPD_DEF"]:
        if c in pos.columns:
            agg_dict[c] = ["max", "mean"]

    base = pos.groupby("SK_ID_CURR").size().to_frame("POS_N_ROWS")

    if agg_dict:
        num_agg = pos.groupby("SK_ID_CURR")[list(agg_dict.keys())].agg(agg_dict)
        num_agg.columns = ["POS_" + "_".join(col).upper() for col in num_agg.columns.ravel()]
        base = base.merge(num_agg, left_index=True, right_index=True, how="left")

    base.reset_index(inplace=True)
    return base

def aggregate_cc(cc_path, prev_path=None):
    if not os.path.exists(cc_path):
        return None
    cc = pd.read_csv(cc_path)

    prev_df = None
    if prev_path and os.path.exists(prev_path):
        prev_df = pd.read_csv(prev_path)[["SK_ID_PREV", "SK_ID_CURR"]]
    cc = ensure_sk_id_curr(cc, left_df=prev_df, left_key="SK_ID_PREV", right_key="SK_ID_PREV")

    if "SK_ID_CURR" not in cc.columns:
        return None

    agg_dict = {}
    for c in ["AMT_BALANCE", "AMT_CREDIT_LIMIT_ACTUAL", "AMT_DRAWINGS_CURRENT",
              "AMT_PAYMENT_CURRENT", "AMT_RECEIVABLE_PRINCIPAL", "AMT_TOTAL_RECEIVABLE",
              "CNT_DRAWINGS_CURRENT", "SK_DPD", "SK_DPD_DEF"]:
        if c in cc.columns:
            agg_dict[c] = ["mean", "max"]

    base = cc.groupby("SK_ID_CURR").size().to_frame("CC_N_ROWS")
    if agg_dict:
        num_agg = cc.groupby("SK_ID_CURR")[list(agg_dict.keys())].agg(agg_dict)
        num_agg.columns = ["CC_" + "_".join(col).upper() for col in num_agg.columns.ravel()]
        base = base.merge(num_agg, left_index=True, right_index=True, how="left")

    # Utilization proxy: mean(balance / limit)
    if "AMT_BALANCE" in cc.columns and "AMT_CREDIT_LIMIT_ACTUAL" in cc.columns:
        tmp = cc.copy()
        tmp["CC_UTIL"] = ratio(tmp["AMT_BALANCE"], tmp["AMT_CREDIT_LIMIT_ACTUAL"])
        util = tmp.groupby("SK_ID_CURR")["CC_UTIL"].agg(["mean", "max"]).add_prefix("CC_UTIL_")
        base = base.merge(util, left_index=True, right_index=True, how="left")

    base.reset_index(inplace=True)
    return base

def aggregate_installments(inst_path, prev_path=None):
    if not os.path.exists(inst_path):
        return None
    ins = pd.read_csv(inst_path)

    # Pastikan punya SK_ID_CURR
    prev_df = None
    if prev_path and os.path.exists(prev_path):
        prev_df = pd.read_csv(prev_path)[["SK_ID_PREV", "SK_ID_CURR"]]
    ins = ensure_sk_id_curr(ins, left_df=prev_df, left_key="SK_ID_PREV", right_key="SK_ID_PREV")

    if "SK_ID_CURR" not in ins.columns:
        return None

    # Fitur pembayaran
    if "AMT_PAYMENT" in ins.columns and "AMT_INSTALMENT" in ins.columns:
        ins["PAYMENT_PERC"] = ratio(ins["AMT_PAYMENT"], ins["AMT_INSTALMENT"])
        ins["PAYMENT_DIFF"] = pd.to_numeric(ins["AMT_INSTALMENT"], errors="coerce") - pd.to_numeric(ins["AMT_PAYMENT"], errors="coerce")
    if "DAYS_ENTRY_PAYMENT" in ins.columns and "DAYS_INSTALMENT" in ins.columns:
        delta = pd.to_numeric(ins["DAYS_ENTRY_PAYMENT"], errors="coerce") - pd.to_numeric(ins["DAYS_INSTALMENT"], errors="coerce")
        ins["DAYS_PAST_DUE"] = delta.clip(lower=0)
        ins["DAYS_BEFORE_DUE"] = (-delta).clip(lower=0)

    agg_dict = {}
    for c in ["PAYMENT_PERC", "PAYMENT_DIFF", "DAYS_PAST_DUE", "DAYS_BEFORE_DUE", "AMT_PAYMENT"]:
        if c in ins.columns:
            agg_dict[c] = ["mean", "max"]

    base = ins.groupby("SK_ID_CURR").size().to_frame("INST_N_ROWS")
    if agg_dict:
        num_agg = ins.groupby("SK_ID_CURR")[list(agg_dict.keys())].agg(agg_dict)
        num_agg.columns = ["INST_" + "_".join(col).upper() for col in num_agg.columns.ravel()]
        base = base.merge(num_agg, left_index=True, right_index=True, how="left")

    # Count telat >0 dan >30
    if "DAYS_PAST_DUE" in ins.columns:
        grp = ins.groupby("SK_ID_CURR")["DAYS_PAST_DUE"]
        base["INST_CNT_LATE_GT0"]  = grp.apply(lambda s: (s > 0).sum())
        base["INST_CNT_LATE_GT30"] = grp.apply(lambda s: (s > 30).sum())

    base.reset_index(inplace=True)
    return base

print("[LOAD/AGG] bureau & bureau_balance ...")
agg_bureau = aggregate_bureau_and_balance(PATH_BUREAU, PATH_BB)

print("[LOAD/AGG] previous_application ...")
agg_prev = aggregate_previous(PATH_PREV)

print("[LOAD/AGG] POS_CASH_balance ...")
agg_pos = aggregate_pos(PATH_POS, prev_path=PATH_PREV)

print("[LOAD/AGG] credit_card_balance ...")
agg_cc = aggregate_cc(PATH_CC, prev_path=PATH_PREV)

print("[LOAD/AGG] installments_payments ...")
agg_inst = aggregate_installments(PATH_INST, prev_path=PATH_PREV)

# Merge semua agregasi ke app_all
def left_merge_on_sk(df_base, addon):
    if addon is None or len(addon) == 0:
        return df_base
    return df_base.merge(addon, on="SK_ID_CURR", how="left")

app_all = left_merge_on_sk(app_all, agg_bureau)
app_all = left_merge_on_sk(app_all, agg_prev)
app_all = left_merge_on_sk(app_all, agg_pos)
app_all = left_merge_on_sk(app_all, agg_cc)
app_all = left_merge_on_sk(app_all, agg_inst)

# Pisahkan kembali train/test
app_all.sort_values(["__IS_TRAIN__", "SK_ID_CURR"], inplace=True)
train_mask = app_all["__IS_TRAIN__"] == 1
test_mask  = app_all["__IS_TRAIN__"] == 0

X_all = app_all.drop(columns=["__IS_TRAIN__"])
X_train = X_all.loc[train_mask].copy()
X_test  = X_all.loc[test_mask].copy()

# Align kolom train/test (jaga-jaga OHE)
X_train, X_test = X_train.align(X_test, join="left", axis=1, fill_value=np.nan)

# Drop kolom non-fitur
drop_cols = ["SK_ID_CURR"]
X_train = X_train.drop(columns=[c for c in drop_cols if c in X_train.columns])
X_test  = X_test.drop(columns=[c for c in drop_cols if c in X_test.columns])

[LOAD/AGG] bureau & bureau_balance ...
[LOAD/AGG] previous_application ...
[LOAD/AGG] POS_CASH_balance ...
[LOAD/AGG] credit_card_balance ...
[LOAD/AGG] installments_payments ...


In [ ]:
# ============================================================
# 4) CV Training — LGBM (wajib) + CatBoost (opsional) + Blend
# ============================================================
# === PATCH: Sanitasi nama kolom agar aman untuk LightGBM ===
import re

def sanitize_cols(cols):
    seen = {}
    new_cols = []
    for c in cols:
        # ganti karakter non [A-Za-z0-9_] menjadi underscore
        nc = re.sub(r'[^A-Za-z0-9_]', '_', str(c))
        # kompres multiple underscore -> single underscore
        nc = re.sub(r'_{2,}', '_', nc)
        # hapus underscore di awal/akhir
        nc = nc.strip('_')
        # kalau kosong, beri nama default
        if nc == '':
            nc = 'COL'
        # jaga keunikan nama
        base = nc
        k = 1
        while nc in seen:
            k += 1
            nc = f"{base}_{k}"
        seen[nc] = 1
        new_cols.append(nc)
    return new_cols

# Terapkan ke X_train dan X_test *dengan cara konsisten*
orig_cols = list(X_train.columns)
sanitized = sanitize_cols(orig_cols)

rename_map = dict(zip(orig_cols, sanitized))
X_train = X_train.rename(columns=rename_map)
X_test  = X_test.rename(columns=rename_map)

# Pastikan tipe 'category' tetap terdeteksi setelah rename
cat_cols = [c for c in X_train.columns if str(X_train[c].dtype) == "category"]

# Opsional: deteksi kalau masih ada karakter terlarang (debug)
bad = [c for c in X_train.columns if re.search(r'["\\\/{}\[\]:,\n\r\t]', c)]
if bad:
    print("Masih ada kolom bermasalah:", bad)

print("[CV] Start 5-fold training ...")
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)

oof_lgb = np.zeros(len(X_train))
oof_cat = np.zeros(len(X_train)) if HAS_CATBOOST else None

models_lgb = []
models_cat = []
best_iters_lgb = []

# class imbalance ratio
pos_rate = TARGET.mean()
neg = (TARGET == 0).sum()
pos = (TARGET == 1).sum()
scale_pos_weight = max(1.0, neg / max(1, pos))

# kandidat kolom kategori (yang masih category dtype) — LightGBM bisa menanganinya
cat_cols = [c for c in X_train.columns if str(X_train[c].dtype) == "category"]

fold_idx = 0
for tr_idx, va_idx in skf.split(X_train, TARGET):
    fold_idx += 1
    X_tr, X_va = X_train.iloc[tr_idx], X_train.iloc[va_idx]
    y_tr, y_va = TARGET[tr_idx], TARGET[va_idx]

    # ---- LightGBM
    dtr = lgb.Dataset(X_tr, label=y_tr, categorical_feature=cat_cols if cat_cols else "auto", free_raw_data=True)
    dva = lgb.Dataset(X_va, label=y_va, categorical_feature=cat_cols if cat_cols else "auto", reference=dtr, free_raw_data=True)

    params_lgb = {
        "objective": "binary",
        "metric": "auc",
        "learning_rate": 0.05,
        "num_leaves": 127,
        "feature_fraction": 0.8,
        "bagging_fraction": 0.8,
        "bagging_freq": 3,
        "min_data_in_leaf": 60,
        "lambda_l1": 0.1,
        "lambda_l2": 0.1,
        "verbosity": -1,
        "force_col_wise": True,
        "scale_pos_weight": scale_pos_weight,
        "seed": RANDOM_STATE,
    }

    model_lgb = lgb.train(
        params_lgb,
        dtr,
        num_boost_round=5000,
        valid_sets=[dtr, dva],
        valid_names=["train", "valid"],
        callbacks=[lgb.early_stopping(EARLY_STOPPING_ROUNDS), lgb.log_evaluation(0)]
    )
    best_iters_lgb.append(model_lgb.best_iteration)
    models_lgb.append(model_lgb)
    oof_lgb[va_idx] = model_lgb.predict(X_va, num_iteration=model_lgb.best_iteration)

    # ---- CatBoost (opsional)
    if HAS_CATBOOST:
        cat_idx = [X_train.columns.get_loc(c) for c in cat_cols] if cat_cols else []
        train_pool = Pool(X_tr, y_tr, cat_features=cat_idx if cat_idx else None)
        valid_pool = Pool(X_va, y_va, cat_features=cat_idx if cat_idx else None)
        model_cat = CatBoostClassifier(
            loss_function="Logloss",
            eval_metric="AUC",
            depth=6,
            learning_rate=0.05,
            l2_leaf_reg=3.0,
            iterations=10000,
            random_seed=RANDOM_STATE,
            od_type="Iter",
            od_wait=300,
            verbose=False,
            class_weights=[1.0, float(scale_pos_weight)]
        )
        model_cat.fit(train_pool, eval_set=valid_pool, verbose=False)
        models_cat.append(model_cat)
        oof_cat[va_idx] = model_cat.predict_proba(X_va)[:, 1]

    # ---- Metrics per fold
    auc_lgb = roc_auc_score(y_va, oof_lgb[va_idx])
    ap_lgb  = average_precision_score(y_va, oof_lgb[va_idx])
    msg = f"[Fold {fold_idx}] LGBM AUC={auc_lgb:.5f}  PR-AUC={ap_lgb:.5f}"
    if HAS_CATBOOST:
        auc_cat = roc_auc_score(y_va, oof_cat[va_idx])
        ap_cat  = average_precision_score(y_va, oof_cat[va_idx])
        msg += f" | CAT AUC={auc_cat:.5f} PR-AUC={ap_cat:.5f}"
    print(msg)

# OOF metrics
print("\n[OOF] LGBM AUC: %.6f | PR-AUC: %.6f" % (roc_auc_score(TARGET, oof_lgb), average_precision_score(TARGET, oof_lgb)))
if HAS_CATBOOST:
    print("[OOF] CAT  AUC: %.6f | PR-AUC: %.6f" % (roc_auc_score(TARGET, oof_cat), average_precision_score(TARGET, oof_cat)))


[CV] Start 5-fold training ...
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[176]	train's auc: 0.919806	valid's auc: 0.776709
[Fold 1] LGBM AUC=0.77671  PR-AUC=0.26671
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[167]	train's auc: 0.914739	valid's auc: 0.785494
[Fold 2] LGBM AUC=0.78549  PR-AUC=0.27707
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[208]	train's auc: 0.931875	valid's auc: 0.779459
[Fold 3] LGBM AUC=0.77946  PR-AUC=0.26776
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[226]	train's auc: 0.937661	valid's auc: 0.783815
[Fold 4] LGBM AUC=0.78382  PR-AUC=0.27406
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[165]	train's auc: 0.914439	valid's auc: 0.777148
[Fold 5] LGBM AUC=0.77715  PR-AUC=0.26580

[OOF] LGBM AUC: 0.78041

In [ ]:
# ============================================================
# 5) BLENDING (simple average, atau hanya LGBM jika CatBoost tidak ada)
# ============================================================

if HAS_CATBOOST:
    # Cari bobot blend terbaik secara grid kecil
    ws = np.linspace(0.0, 1.0, 11)
    best_w, best_auc, best_ap = 0.5, -1, -1
    for w in ws:
        blend = w * oof_lgb + (1 - w) * oof_cat
        auc = roc_auc_score(TARGET, blend)
        ap  = average_precision_score(TARGET, blend)
        if auc > best_auc:
            best_auc, best_ap, best_w = auc, ap, w
    print(f"[BLEND] Best w(LGBM)={best_w:.2f} -> AUC={best_auc:.6f} PR-AUC={best_ap:.6f}")
    oof_blend = best_w * oof_lgb + (1 - best_w) * oof_cat
else:
    best_w = 1.0
    oof_blend = oof_lgb
    best_auc = roc_auc_score(TARGET, oof_blend)
    best_ap  = average_precision_score(TARGET, oof_blend)
    print(f"[BLEND] Using LGBM only -> AUC={best_auc:.6f} PR-AUC={best_ap:.6f}")


[BLEND] Using LGBM only -> AUC=0.780410 PR-AUC=0.269932


In [ ]:
# ============================================================
# 6) CALIBRATION (Isotonic) di OOF -> mempelajari mapping p->p_cal
# ============================================================

print("[CALIB] Fitting IsotonicRegression on OOF blend ...")
calibrator = IsotonicRegression(out_of_bounds="clip")
calibrator.fit(oof_blend, TARGET)

[CALIB] Fitting IsotonicRegression on OOF blend ...


IsotonicRegression(out_of_bounds='clip')

In [ ]:
# ============================================================
# 7) FINAL REFIT on FULL TRAIN lalu PREDICT TEST
# ============================================================

print("[REFIT] Train final LGBM on full data ...")
# num_boost_round pakai rata-rata best_iter
final_num_boost_round = int(np.mean(best_iters_lgb)) if best_iters_lgb else 1000

dall = lgb.Dataset(X_train, label=TARGET, categorical_feature=cat_cols if cat_cols else "auto")
final_lgb = lgb.train(
    {**params_lgb, "verbosity": -1},
    dall,
    num_boost_round=final_num_boost_round
)

# CatBoost final (opsional)
final_cat = None
if HAS_CATBOOST:
    cat_idx = [X_train.columns.get_loc(c) for c in cat_cols] if cat_cols else []
    pool_all = Pool(X_train, TARGET, cat_features=cat_idx if cat_idx else None)
    final_cat = CatBoostClassifier(
        loss_function="Logloss",
        eval_metric="AUC",
        depth=6,
        learning_rate=0.05,
        l2_leaf_reg=3.0,
        iterations=int(np.mean([m.tree_count_ for m in models_cat])) if models_cat else 2000,
        random_seed=RANDOM_STATE,
        verbose=False,
        class_weights=[1.0, float(scale_pos_weight)]
    )
    final_cat.fit(pool_all, verbose=False)

# Prediksi test -> blend -> calibrate
print("[PREDICT] Inference on test ...")
pred_lgb_test = final_lgb.predict(X_test, num_iteration=final_num_boost_round)

if HAS_CATBOOST and final_cat is not None:
    pred_cat_test = final_cat.predict_proba(X_test)[:, 1]
    pred_blend_test = best_w * pred_lgb_test + (1 - best_w) * pred_cat_test
else:
    pred_blend_test = pred_lgb_test

pred_cal_test = calibrator.predict(pred_blend_test)


[REFIT] Train final LGBM on full data ...
[PREDICT] Inference on test ...


In [ ]:
# ============================================================
# 8) SAVE artifacts & submission
# ============================================================

sub = pd.DataFrame({
    "SK_ID_CURR": SK_ID_TEST,
    "TARGET": pred_cal_test
})
sub_path = os.path.join(OUT_DIR, "submission.csv")
sub.to_csv(sub_path, index=False)
print(f"[OK] Submission saved to: {sub_path}")

# Save metadata ringkas
meta = {
    "n_splits": N_SPLITS,
    "random_state": RANDOM_STATE,
    "scale_pos_weight": float(scale_pos_weight),
    "oof_auc_lgb": float(roc_auc_score(TARGET, oof_lgb)),
    "oof_pr_auc_lgb": float(average_precision_score(TARGET, oof_lgb)),
    "oof_auc_blend": float(roc_auc_score(TARGET, oof_blend)),
    "oof_pr_auc_blend": float(average_precision_score(TARGET, oof_blend)),
    "best_w_lgb": float(best_w),
    "final_num_boost_round_lgb": int(final_num_boost_round),
    "has_catboost": bool(HAS_CATBOOST),
}
with open(os.path.join(OUT_DIR, "meta.json"), "w") as f:
    json.dump(meta, f, indent=2)
print(f"[OK] Meta saved to: {os.path.join(OUT_DIR, 'meta.json')}")

[OK] Submission saved to: /content/hci_outputs/submission.csv
[OK] Meta saved to: /content/hci_outputs/meta.json
